# Omoju Miller
# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?
- This sounds like a classic classification problem because we are trying to classify our data into two separate classes.
   - The inherent task is "predicting a category."
- Another support we have in evaluating this problem as a classification problem is that we can consider our data as labeled. We have features that we can use to determine who has succeeded in the class versus who has not. For that insight, we could use `passed` column as our class label

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.


In [3]:
# Import libraries
import numpy as np
import pandas as pd

In [4]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [5]:
from __future__ import division

# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns) - 1
n_passed = len(student_data.query('passed == "yes"'))
n_failed = len(student_data.query('passed == "no"'))
grad_rate = n_passed / n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 0.67%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [6]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-", feature_cols

print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):- ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [46]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))


Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [8]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

In [9]:
from sklearn import cross_validation



def shuffle_split_data(X, y):
    """ Shuffles and splits data into 75% training and 25% testing subsets,
        then returns the training and testing subsets. """

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.24)
    


    # Return the training and testing data subsets
    return X_train, y_train, X_test, y_test




In [10]:
# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

# Test shuffle_split_data
try:
    X_train, y_train, X_test, y_test = shuffle_split_data(X_all, y_all)
    print "Successfully shuffled and split the data!"
except:
    print "Something went wrong with shuffling and splitting the data."


print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Successfully shuffled and split the data!
Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [11]:
# Train a model
import time
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    return end - start

# Predict on training set and compute F1 score

def predict_labels(clf, features, target):
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    return f1_score(target.values, y_pred, pos_label='yes'), end - start

In [12]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    
    train_time = train_classifier(clf, X_train, y_train)
    
    F1_score_train, prediction_time_train = predict_labels(clf, X_train, y_train)
    F1_score_test, prediction_time_test = predict_labels(clf, X_test, y_test)
    
    return train_time, F1_score_train, F1_score_test, prediction_time_test



In [13]:
def print_Output_Table(clf, train_time, prediction_time, F1_score_train, F1_score_test):

    print "{:>53}".format(clf.__class__.__name__)
    print "{:>53}".format('Training set size')
    print "{:>30}{:>d}\t{:>d}\t{:>d}".format('',100,200,300)
    print "-"*55

    for i in range(1):
        print "{:30}{:.3f}\t{:.3f}\t{:.3f}".format('Training time (secs)',
                                           train_time[i], train_time[i+1], train_time[i+2])
        print "{:30}{:.3f}\t{:.3f}\t{:.3f}".format('Prediction time (secs)',
                                           prediction_time[i], prediction_time[i+1], prediction_time[i+2])
        print "{:30}{:.3f}\t{:.3f}\t{:.3f}".format('F1 score for training set',
                                          F1_score_train[i], F1_score_train[i+1],F1_score_train[i+2])
        print "{:30}{:.3f}\t{:.3f}\t{:.3f}".format('F1 score for test set',
                                          F1_score_test[i],F1_score_test[i+1],F1_score_test[i+2])

In [14]:

# print output for training on the full training size
def makeTableFullTrainingSize(train_time, F1_score_train, F1_score_test, prediction_time_test):
    print "{:>35}".format(clf.__class__.__name__)
    print "-"*35
    print "{:30}{:.3f}".format('Training time (secs)', train_time)
    print "{:30}{:.3f}".format('Prediction time (secs)', prediction_time_test)
    print "{:30}{:.3f}".format('F1 score for training set', F1_score_train)
    print "{:30}{:.3f}".format('F1 score for test set', F1_score_test)


# Lets try incremements of 100, 200 and 300
def makeTable(clf, X_test, y_test):

    train_set_size = [0] * 3
    train_time = [0] * 3
    prediction_time = [0] * 3
    F1_score_train = [0] * 3
    F1_score_test = [0] * 3

    for i in xrange(100, 400, 100):
        (train_time[(i//100)-1], F1_score_train[(i//100)-1], 
         F1_score_test[(i//100)-1], prediction_time[(i//100)-1]) = train_predict(clf, 
                                                              X_train[:i], y_train[:i], X_test, y_test)
        train_set_size[(i//100)-1] = i


    print_Output_Table(clf, train_time, prediction_time, F1_score_train, F1_score_test)

In [78]:
## Confusion Matrix
## Predicted Class by Actual Class
from sklearn.metrics import confusion_matrix

def makeConfusionMatrix(X_test, y_test):
    y_true = y_test
    y_pred = clf.predict(X_test)

    cM = confusion_matrix(y_true, y_pred)

    print "{:^}".format(clf.__class__.__name__)
    print "{:>40}".format('Actual Class')
    print "{:15}{:10}{:10}{:10}".format('Predicted', '', 'Passed', 'Failed')
    print "{:15}{:10}{:<10.3f}{:<10.3f}".format('', 'Passed', cM[0][0], cM[0][1])
    print "{:15}{:10}{:<10.3f}{:<10.3f}".format('', 'Failed', cM[1][0], cM[1][1])

### Dictionary of models to run


In [83]:
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier




models = {'RandomForest': RandomForestClassifier(n_estimators=200),
          'SVC': svm.SVC(kernel="linear"),
          'DecisionTree': tree.DecisionTreeClassifier()
         }

In [85]:


for model_name, clf in models.iteritems():
# Fit model to the training data, and predict labels (for both training and test sets), and measure the F1 score. 
    train_classifier(clf, X_train, y_train)
    train_time, F1_score_train, F1_score_test, prediction_time_test = train_predict(clf, X_train, y_train, X_test, y_test)
    makeTableFullTrainingSize(train_time, F1_score_train, F1_score_test, prediction_time_test)

    print "\n"

    # Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.
    makeTable(clf, X_test, y_test)

    print "\n"
    # Create confusion matrix
    makeConfusionMatrix(X_test, y_test)
    
    print "\n"

             RandomForestClassifier
-----------------------------------
Training time (secs)          0.390
Prediction time (secs)        0.012
F1 score for training set     1.000
F1 score for test set         0.769


                               RandomForestClassifier
                                    Training set size
                              100	200	300
-------------------------------------------------------
Training time (secs)          0.406	0.379	0.386
Prediction time (secs)        0.011	0.012	0.013
F1 score for training set     1.000	1.000	1.000
F1 score for test set         0.739	0.763	0.786


                            Actual Class
Predicted                Passed    Failed    
               Passed    10.000    25.000    
               Failed    5.000     55.000    
                                SVC
-----------------------------------
Training time (secs)          0.018
Prediction time (secs)        0.001
F1 score for training set     0.852
F1 score for test set  

In [34]:
from sklearn.metrics import f1_score
from sklearn.cross_validation import cross_val_score
from sklearn.learning_curve import learning_curve


## What is the implication?

Remember what we are trying to do here? We are trying to identifu students who need intervention, students who left to their own devices will **not** pass the class. Given our task, the cost of false alarms (12) aren't that high in comparison to **missing** students that need an intervention (23).



## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [36]:
# TODO: Fine-tune your model and report the best F1 score


Best estimator found through GridSearch SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
  
F1 score for training set: 0.855835240275

F1 score for test set: 0.75

## Validating the Model

In [39]:
clf = svm.SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)



In [40]:
makeTable(clf, X_test, y_test)

                                                  SVC
                                    Training set size
                              100	200	300
-------------------------------------------------------
Training time (secs)          0.056	0.269	0.350
Prediction time (secs)        0.001	0.001	0.001
F1 score for training set     0.958	0.858	0.853
F1 score for test set         0.581	0.656	0.733


In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn import svm


print("Fitting the classifier to the training set")
t0 = time()
parameters = {'kernel':('linear', 'rbf'), 
              'C':[1e1, 5e1, 1e2, 5e2, 1e3, 5e3, 1e4, 5e4],
             'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]}
svr = svm.SVC()
clf = GridSearchCV(svr, parameters)

# Fit model to training data

train_classifier(clf, X_train, y_train) 
print("done in %0.3fs" % (time() - t0))

print"Best estimator found by grid search: {}".format((clf.best_estimator_))
    
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))